
#### Part A\
\
Load the titanic.csv file into a data frame. Are there any missing values in the
data frame? How many missing values occur for each of the columns?


In [124]:
# 2.a

my_data <- read.csv( "/public/bmort/R/titanic.csv")

my_data[my_data == 'NA'] <- NA
my_data[my_data == ''] <- NA
my_data[my_data == 'NaN'] <- NA

sum(is.na(my_data$PassengerId))
sum(is.na(my_data$Survived))
sum(is.na(my_data$Pclass))
sum(is.na(my_data$Name))
sum(is.na(my_data$Sex))
sum(is.na(my_data$Age))
sum(is.na(my_data$SibSp))
sum(is.na(my_data$Parch))
sum(is.na(my_data$Fare))
sum(is.na(my_data$Cabin))
# sum(is.na(my_data$Embarked))

my_data[my_data$Survived == 1, ]$Survived <- "Yes"
my_data[my_data$Survived == 0, ]$Survived <- "No"

# convert survived data into factor data
my_data$Survived <- as.factor(my_data$Survived)
my_data$Pclass <- as.factor(my_data$Pclass)

my_data[is.na(my_data)] <- NA
nrow(my_data[is.na(my_data$Survived) | 
             is.na(my_data$Age) | 
             is.na(my_data$Pclass), ])


# removing all NA values
my_data <- my_data[!(is.na(my_data$Survived) |
                    is.na(my_data$Age) |
                    is.na(my_data$Pclass)), ]



[1] 0

[1] 0

[1] 0

[1] 0

[1] 0

...

[1] 0

[1] 0

[1] 0

[1] 687

[1] 177


The column "Age" has 177 missing values. The column "Cabin" has 687 missing values.



#### Part B\
\
What percent of the passengers survived?

In [125]:
# 2.b

n <- length(my_data$Survived)
survivor <- length(my_data$Survived[my_data$Survived == 'Yes'])
percent <- survivor / n
print(percent)

[1] 0.4061625



Approximately $41.2\%$ of passengers survived.



#### Part C\
\
What was the most amount of money that was paid to purchase a ticket by a
passenger?



In [126]:
# 2.c

print(max(my_data$Fare))

[1] 512.3292



The most amount of money that was paid to purchase a ticket was $512.33.



#### Part D\
\
How many unique places did the passengers embark from?


In [127]:
# 2.d
print(length(unique(my_data$Embarked)))

[1] 4


In [ ]:

There are four unique places passengers embarked from.



#### Part E\
\
Partition the titanic data set so that 80% will be used for training and 20% will be
used for testing your machine learning model. You can do the partition manually at
random or use the createDataPartition() function in R's caret library. 

In [128]:
# 2.e
library("caret")
set.seed(123)

# partition variables 
x1 <-my_data$Age
x2 <- my_data$Pclass
y <- my_data$Survived


uni_partition <- createDataPartition(x1, times = 1,  p = 0.8, list = F)
x1_partition <- x1[uni_partition]
x2_partition <- x2[uni_partition]

x1_partition_test <- x1[-uni_partition]
x2_partition_test <- x2[-uni_partition]

# y data partitioning
y_train <- y[uni_partition]
y_test <- y[-uni_partition]

new_df <- data.frame(x1_partition, x2_partition, y_train)
colnames(new_df) <- c("Age", "Pclass", "Survived")
new_df_test <- data.frame(x1_partition_test, x2_partition_test, y_test)
colnames(new_df_test) <- c("Age", "Pclass", "Survived")



#### Part F\
\
Use a logistic regression method from R's caret library to generate a machine
learning model to predict whether a passenger will survive based on the age and
class of the passenger. Using the caret library's trainControl() function, check
your model parameter and feature selection by performing repeated crossvalidation (with 5-folds) on the training data for your model. Consult the caret
library documentation as needed. 

In [129]:
# part f
library("dplyr")
library("caret")
set.seed(1)

library(caret)

control <- trainControl(
    method = "repeatedcv",
    number = 5,
    repeats = 3,  
  )

model <- train(
  Survived ~ Age + Pclass,
  data = new_df,
  method = "glm",
  family = "binomial",
  trControl = control
)

print(summary(model))


Call:
NULL

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.9048  -0.8384  -0.5879   0.9728   2.4838  

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept)  2.486643   0.361407   6.880 5.97e-12 ***
Age         -0.047241   0.007657  -6.169 6.86e-10 ***
Pclass2     -1.024669   0.265236  -3.863 0.000112 ***
Pclass3     -2.548261   0.273074  -9.332  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 775.73  on 572  degrees of freedom
Residual deviance: 656.94  on 569  degrees of freedom
AIC: 664.94

Number of Fisher Scoring iterations: 4




#### Part G\ 
\
Use the test data set (i.e. the 20% of the data that was kept aside earlier) to
generate a final validation for your model with the predict() function in the
caret library. Generate a confusion matrix and comment on the accuracy of the
model. 

In [130]:
# part g

predictions <- predict(model, newdata = new_df_test)

# Create a confusion matrix
conf_matrix <- confusionMatrix(predictions, new_df_test$Survived)

# Print the confusion matrix
print(conf_matrix)

Confusion Matrix and Statistics

          Reference
Prediction No Yes
       No  63  24
       Yes 23  31
                                          
               Accuracy : 0.6667          
                 95% CI : (0.5824, 0.7437)
    No Information Rate : 0.6099          
    P-Value [Acc > NIR] : 0.09682         
                                          
                  Kappa : 0.2972          
                                          
 Mcnemar's Test P-Value : 1.00000         
                                          
            Sensitivity : 0.7326          
            Specificity : 0.5636          
         Pos Pred Value : 0.7241          
         Neg Pred Value : 0.5741          
             Prevalence : 0.6099          
         Detection Rate : 0.4468          
   Detection Prevalence : 0.6170          
      Balanced Accuracy : 0.6481          
                                          
       'Positive' Class : No              
                                 

The confusion matrix states the logistic regression is about $66.\bar{6}\%$ accurate in predicting a conclusion based on the data inserted. This entails there is a $33.\bar{3}\%$ error rate. 


#### Part H\
\
Based on your model, classify the passengers provided in the unlabeled /public/bmort/R/titanic-unknown.csv data set. Indicate which passengers in this data set survived. How confident are you in the answer? Explain your reasoning.

In [131]:
unknown <- read.csv("/public/bmort/R/titanic-unknown.csv")
unknown$Pclass <- as.factor(unknown$Pclass)
predict_unknown <- predict(model, newdata = unknown)
print(predict_unknown)


[1] No  Yes No  Yes No 
Levels: No Yes



After classifying the unknown passengers provided in the unlabeled data set I found that two passengers are predicted to survive and three are not. I am not very confident in the results of this logistical regression. There are a lot of missing values in the data set which makes having an accurate prediction difficult. Because some the ages were missing there was no prediction for their survival.
